In [1]:
import midi2nmat as m2n
import pandas as pds
import numpy as np

In [2]:
path="/Users/wsdula/Documents/ATAVizM/Data/a-exsultate.mid"

In [3]:
test = m2n.midi2nmat(path)

In [4]:
def SAR_Contour(nmat, s_type=3, cmin=5):
    cmax=50
    degrees=2
    pitches=list(nmat['Pitch'])
    contcom = _make_ContCM(pitches,degrees)
    CL=None #This will be a list of column sums of contcom
    
    
    durs=list(nmat['Time'])
    durs=durs[1:len(durs)-1]-durs[0:len(durs)-2]
    durs.append(nmat['Duration'].last())
    contcom=_make_ContCM(durs,degrees)
    DCL=None #This will be a list of column sums of contcom
    
    

In [18]:
def _make_ContCM(arr, deg1, deg2 = None):
    """
    Creates Continous Contour Matrix for searching
    Adapted for asynchrony
    """
    
    if deg2 is None:
        deg2 = deg1
    
    degrees = deg1 + deg2
    ContCM = np.zeros((degrees,len(arr))) #creates and initializes  contour matrix w/ zeroes
    ContCM.fill(np.nan)            #fills matrix with Null values
    print(arr)
    for n in range(0,len(arr)):    
            for i in [1,2]:
                try:
                    if arr[n] > arr[n-i] and n-i >= 0 :
                        ContCM[deg1-i, n] = 1
                    elif n-i >= 0:
                        ContCM[deg1-i, n] = 0
                except:
                    continue

            for i in [1,2]:
                try:
                    if arr[n] > arr[n+i]:
                        ContCM[deg1+i-1, n] = 1
                    else:
                        ContCM[deg1+i-1, n] = 0
                except:
                    continue
    
    return ContCM

In [19]:
SAR_Contour(test)

[60, 60, 65, 64, 65, 67, 67, 60, 60, 65, 64, 65, 67, 67, 65, 67, 64, 67, 69, 65, 69, 67, 69, 70, 67, 64, 62, 64, 65, 67, 65, 60, 65, 65, 65, 62, 65, 64, 67, 65, 62, 60, 62, 64, 62, 60, 60, 62, 60, 60, 65, 64, 65, 67, 67, 69, 65, 64, 65, 67, 67, 67, 67, 67, 70, 67, 69, 70, 69, 67, 67, 69, 65, 64, 65, 67, 67, 69, 65, 67, 65, 64, 65, 64, 64, 65, 64, 64, 65, 67, 69, 69, 67, 69, 65, 62, 67, 67, 65, 64, 62, 60, 62, 60, 60, 62, 64, 65, 67, 67, 69, 65, 64, 65, 67, 60, 60, 65, 64, 65, 67, 67, 69, 70, 72, 70, 67, 69, 70, 69, 70, 69, 67, 69, 67, 67, 67, 70, 67, 69, 70, 69, 70, 69, 67, 69, 67, 67, 62, 62, 65, 64, 65, 67, 67, 65, 67, 64, 67, 69, 65, 69, 67, 69, 70, 67, 64, 62, 64, 65, 67, 65, 60, 65, 65, 65, 62, 65, 64, 67, 65, 62, 60, 62, 64, 62, 60, 60, 60, 65, 64, 65, 67, 67, 65, 67, 64, 67, 69, 65, 69, 67, 69, 70, 67, 64, 62, 64, 65, 67, 65, 60, 65, 65, 65, 62, 65, 64, 67, 65, 62, 60, 62, 64, 62, 60]
[[nan nan  1.  1.  0.  1.  1.  0.  0.  1.  1.  0.  1.  1.  0.  0.  0.  0.
   1.  0.  0.  1.  0.